# C# Features

| C#    | SDK    |
| :---- | :----- |
| C# 12 | .NET 8 |
| C# 11 | .NET 7 |
| C# 10 | .NET 6 |

In [3]:
Console.WriteLine($"Running .NET CLR {System.Environment.Version}");

You running .NET CLR 8.0.0


## C# 12
- primary constructors
- collection expressions
- ref readonly parameters
- default lambda parameters
- alias any type
- inline arrays
- experimental attribute
- interceptors (experimental)

https://learn.microsoft.com/en-us/dotnet/csharp/whats-new/csharp-12
https://learn.microsoft.com/en-us/dotnet/core/whats-new/dotnet-8

In [22]:

/*
    C# 12 features

    Primary constructors:
    - parameters are auto private fields
    - Employee's id not publicly accessible
    - need to becareful reusing parameter names
      - both classes have their own copies of id

    convenient to avoid declaring parameters
    might be problematic if there are multiple
    levels of inheritance with primary constructors
    might make it difficult to trace parameters
*/

public class Employee(int id)
{
    public int Id => id;

    public override string ToString() => $"Id: {id}";
}

public class Manager(int id, string rank) : Employee(id)
{
    public string Rank => rank;

    public override string ToString() => $"Id: {id}, Rank: {rank}";

    public void ChangeId(int newId) => id = newId;
}

public class Director : Employee
{
    public Director(int id) : base(id)
    {
        // Director will not have auto private field of id
        // Director class will reuse Employee's Id
    }
}

Employee emp = new(1);
// emp.Id = 5; // read only; no assignment
Console.WriteLine($"Employee Id: {emp.Id}");

Manager mgr = new(5, "assistant");
Console.WriteLine($"Manager: {mgr}");

// both classes have their own copies of id
// updating one doesn't change the other
mgr.ChangeId(20);
Console.WriteLine($"Manager: {mgr}");
Employee emp2 = mgr as Employee;
Console.WriteLine($"Employee: {emp2.Id}"); // still show as 5



Employee Id: 0
Manager: Id: 5, Rank: assistant
Manager: Id: 20, Rank: assistant
Employee: 0


In [20]:
/*
    C# 12 features
    
    Collection expressions
    - spread operator ..
    - creating collections using collections
*/

int[] array = [ 1, 2, 3];
List<int> list = [1, 2, 3];
int[] combined = [..array, ..list];
Console.WriteLine($"Length of combined: {combined.Length}");

// int[][] combined2d = [array, list]; // compiler complains list not correct type
int[][] combined2d = [array, array];
Console.WriteLine($"Combined length: {combined2d.Length}");
Console.WriteLine($"Combined depth: {combined2d[0].Length}");

// this hints that this syntax work by enumerating all values then combining into array 
// todo: check on mem allocations
int[] combined3 = [..array, ..System.Linq.Enumerable.Range(0, 10).Where(x => x % 2 == 0)];
Console.WriteLine($"Combined3: {string.Join(' ', combined3)}")


Length of combined: 6
Combined length: 2
Combined depth: 3
Combined3: 1 2 3 0 2 4 6 8


In [10]:
/*
    C# 12 features
    
    ref readonly
    - works with value types; additionally prevent value's properties from changing
    - reference types don't get the same safety on properties/fields
    - tool to prevent accidental writes on parameters
*/

public class RefClass
{
    public string Name { get; set; } = string.Empty;
}

public struct RefStruct
{
    public string Name { get; set; }
}

public static void ChangeRc(ref readonly RefClass rc)
{
    rc.Name = "rc changed"; // no compiler error
    // rc = new RefClass { Name = "new class" }; // compiler error
    Console.WriteLine($"RefClass name: {rc.Name}");
}

public static void ChangeRs(ref readonly RefStruct rs)
{
    // rs.Name = "rs changed"; // compiler error
    Console.WriteLine($"RefStruct name: {rs.Name}");
}

RefClass rc = new();
ChangeRc(ref rc);
//ChangeRc(ref new RefClass()); // compiler error; "ref" by itself already prevents this


RefClass name: rc changed


In [13]:
/*
    C# 12 features
    
    default lambda parameters
    - improved lambda's dev experienced
    - default values for input parameters
*/

var lambcha = (int x, int y = 0) => 
{
    return x * y;
};

var lambyar = (params string[] words) => 
{
    // just an example, not safe to use
    return string.Join(words[0], words[1..]);
};

Console.WriteLine($"Result: {lambcha(1, 2)}");
Console.WriteLine($"Result: {lambcha(5)}");
Console.WriteLine($"Result: {lambcha(9)}");
Console.WriteLine($"Words: {lambyar(",", "hello", "world")}");

Result: 2
Result: 0
Result: 0
Words: hello,world


In [17]:
/*
    C# 12 features
    
    alias any type
    - i hope this brings more good than harm...
*/

using moo = int;
using hamster = System;
using hamster.Linq;
moo num = 5;


In [20]:
/*
    C# 12 features
    
    inline arrays
    - seems fairly limited
    - based on docs, the perf is similar to unsafe fixed size buffer
    - might be suitable for certain scenarios where stackalloc/Span cannot be used easily
*/

[System.Runtime.CompilerServices.InlineArray(10)]
public struct LargeBuf
{
    private int oneAndOnlyField;
}

var buf = new LargeBuf();
// Span<int> nums = stackalloc int[10]; // compiler error

public static void ShowNumsLength()
{
    Span<int> nums = stackalloc int[10];
    Console.WriteLine($"Nums length: {nums.Length}");
}

ShowNumsLength();



Nums length: 10
